In [35]:
from data import CamVid as dataset
from data.utils import enet_weighing
from models.enet import ENet
from metric.iou import IoU
import utils
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import transforms as ext_transforms
from PIL import Image

In [32]:
# Display the example of model structure

model = ENet(12).to("cuda:0")
optimizer = optim.Adam(model.parameters())
# Load the previoulsy saved model state to the ENet model
model = utils.load_checkpoint(model, optimizer, 'save/eNet_camvid/', 'enet')[0]
print(model)

ENet(
  (initial_block): InitialBlock(
    (main_branch): Conv2d(3, 13, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (ext_branch): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (batch_norm): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (out_prelu): PReLU(num_parameters=1)
  )
  (downsample1_0): DownsamplingBottleneck(
    (main_max1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (ext_conv1): Sequential(
      (0): Conv2d(16, 4, kernel_size=(2, 2), stride=(2, 2), bias=False)
      (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PReLU(num_parameters=1)
    )
    (ext_conv2): Sequential(
      (0): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PReLU(num_parameters=1)
    )
    (ext_conv3)

In [81]:
class Test:
    def __init__(self, model, data_loader, criterion, metric, device):
        self.model = model
        self.data_loader = data_loader
        self.criterion = criterion
        self.metric = metric
        self.device = device

    def run_epoch(self, iteration_loss=False):
        self.model.eval()
        epoch_loss = 0.0
        self.metric.reset()
        for step, batch_data in enumerate(self.data_loader):
            # Get the inputs and labels
            inputs = batch_data[0].to(self.device)
            labels = batch_data[1].to(self.device)

            with torch.no_grad():
                # Forward propagation
                outputs = self.model(inputs)

                # Loss computation
                loss = self.criterion(outputs, labels)

            # Keep track of loss for current epoch
            epoch_loss += loss.item()

            # Keep track of evaluation the metric
            self.metric.add(outputs.detach(), labels.detach())

            if iteration_loss:
                print("[Step: %d] Iteration loss: %.4f" % (step, loss.item()))

        return epoch_loss / len(self.data_loader), self.metric.value()
    
def test(model, test_loader, class_weights, class_encoding):
    print("\nTesting...\n")

    num_classes = len(class_encoding)
    criterion = nn.CrossEntropyLoss(weight=class_weights)

    ignore_index = list(class_encoding).index('unlabeled')

    metric = IoU(num_classes, ignore_index=ignore_index)

    # Test the trained model on the test set
    test = Test(model, test_loader, criterion, metric, device)
    print(">>>> Running test dataset")

    loss, (iou, miou) = test.run_epoch()
    class_iou = dict(zip(class_encoding.keys(), iou))
    print(">>>> Avg. loss: {0:.4f} | Mean IoU: {1:.4f}".format(loss, miou))

    # Print per class IoU
    for key, class_iou in zip(class_encoding.keys(), iou):
        print("{0}: {1:.4f}".format(key, class_iou))
    
    return miou, iou

In [82]:
def test_enet_camvid(save_dir, model_name):
    dataset_dir = 'dataset/camvid/'
    height = 360
    width = 480
    batch_size = 10
    num_classes = 12
    device = "cuda:0"
    
    model = ENet(num_classes).to(device)
    optimizer = optim.Adam(model.parameters())

    # Load the previoulsy saved model state to the ENet model
    model = utils.load_checkpoint(model, optimizer, save_dir, model_name)[0]

    image_transform = transforms.Compose(
        [transforms.Resize((height, width)),
         transforms.ToTensor()])

    label_transform = transforms.Compose([
        transforms.Resize((height, width), Image.NEAREST),
        ext_transforms.PILToLongTensor()
    ])
    # Load the test set as tensors
    test_set = dataset(
        dataset_dir,
        mode='test',
        transform=image_transform,
        label_transform=label_transform)
    test_loader = data.DataLoader(
        test_set,
        batch_size,
        shuffle=False,
        num_workers=12)

    train_set = dataset(
        dataset_dir,
        transform=image_transform,
        label_transform=label_transform)
    train_loader = data.DataLoader(
        train_set,
        batch_size=batch_size,
        shuffle=True,
        num_workers=12)

    # Get encoding between pixel valus in label images and RGB colors
    class_encoding = train_set.color_encoding
    if(len(class_encoding)==13):
        del class_encoding['road_marking']

    class_weights = enet_weighing(train_loader, num_classes)
    if class_weights is not None:
        class_weights = torch.from_numpy(class_weights).float().to(device)
    # Set the weight of the unlabeled class to 0
    ignore_index = list(class_encoding).index('unlabeled')
    class_weights[ignore_index] = 0

    miou, iou = test(model, test_loader, class_weights, class_encoding)
    print('The save_dir is: ',save_dir)
    print('The model_name is: ',model_name)
    return miou, iou

In [84]:
miou_batch10_list = []
iou_batch10_list = []
for i in range(15):
    miou, iou = test_enet_camvid('save/eNet_camvid_batch10_save/','enet_'+str((i+1)*20))
    miou_batch10_list.append(miou)
    iou_batch10_list.append(iou)


Testing...

>>>> Running test dataset
>>>> Avg. loss: 0.9124 | Mean IoU: 0.3843
sky: 0.9011
building: 0.6565
pole: 0.0032
road: 0.8871
pavement: 0.6811
tree: 0.5564
sign_symbol: 0.0000
fence: 0.0792
car: 0.4606
pedestrian: 0.0017
bicyclist: 0.0000
unlabeled: nan
The save_dir is:  save/eNet_camvid_batch10_save/
The model_name is:  enet_20

Testing...

>>>> Running test dataset
>>>> Avg. loss: 0.9927 | Mean IoU: 0.4056
sky: 0.8942
building: 0.4534
pole: 0.1393
road: 0.9005
pavement: 0.7116
tree: 0.4699
sign_symbol: 0.0922
fence: 0.1422
car: 0.5366
pedestrian: 0.1202
bicyclist: 0.0011
unlabeled: nan
The save_dir is:  save/eNet_camvid_batch10_save/
The model_name is:  enet_40

Testing...

>>>> Running test dataset
>>>> Avg. loss: 0.9489 | Mean IoU: 0.4750
sky: 0.9050
building: 0.5925
pole: 0.1751
road: 0.9114
pavement: 0.7264
tree: 0.5832
sign_symbol: 0.1399
fence: 0.1075
car: 0.6871
pedestrian: 0.1807
bicyclist: 0.2165
unlabeled: nan
The save_dir is:  save/eNet_camvid_batch10_save/
The m